# Echtzeit-Datenaufnahme mit Tweepy und Kafka

### Ziele:
- Daten der Twitter-API abfragen 
- Erzeugung eines KafkaProducer für die Austrahlung von Tweet-Streams

### Required libraries

In [8]:
import tweepy
import time
from kafka import KafkaConsumer, KafkaProducer

<a id="1"></a>
### Twitter-API Konfiguration


In [9]:
# twitter setup
consumer_key = "SouT2iNs6847bWSGLNWMvHnZK"
consumer_secret = "5ywQrdlUV9DpFdh6cgjvWN6uKtNEi2h8xtxilUEXJPYSYN5Ly9"
access_token = "917727277-jU7H5KRBRRy5l17NCCAyptJxleZDRrdlti3TlvRe"
access_token_secret = "2zGPcdXvgZkLlQEI4maTt4fJblAQDBwN3cVBJ6OKQ8PqD"
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object by passing in auth information
api = tweepy.API(auth) 


### Normalisierung des Timestamps von GMT auf CEST

In [10]:
from datetime import datetime, timedelta

def normalize_timestamp(time):
    mytime = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    mytime += timedelta(hours=2)   # the tweets are timestamped in GMT timezone, while I am in +1 timezone
    return (mytime.strftime("%Y-%m-%d %H:%M:%S")) 

<a id="2"></a>
### KafkaProducer Konfiguration


In [11]:
producer = KafkaProducer(bootstrap_servers='localhost:9092')
topic_name = 'tweets-lambda1'

<a id="3"></a>
### Producing and sending records to the Kafka Broker
- Daten aus Twitter abfragen und mit Kafka ausstrahlen


In [12]:
def query_twitter_data():
    res = api.search("Big Data OR big data OR iPhone")
    for i in res:
        record = ''
        record += str(i.user.id_str)
        record += ';'
        record += str(normalize_timestamp(str(i.created_at)))
        record += ';'
        record += str(i.user.location)
        record += ';'
        producer.send(topic_name, str.encode(record))

### Funktion für die Abfrage von Tweets 

In [13]:
def stream_data(interval):
    while True:
        query_twitter_data()
        #interval should be an integer, the number of seconds to wait
        time.sleep(interval)


### Neue Daten jede 6 Sekunden von Twitter abfragen und ausstrahlen

In [ ]:
stream_data(60 * 0.1)

### KafkaProducer testen

In [4]:
from kafka import KafkaConsumer
consumer = KafkaConsumer(
                        bootstrap_servers='localhost:9092',
                        auto_offset_reset='latest',  # Reset partition offsets upon OffsetOutOfRangeError
                        group_id='test1',   # must have a unique consumer group id 
                        consumer_timeout_ms=1000)  
                                # How long to listen for messages - we do it for 10 seconds 
                                # because we poll the kafka broker only each couple of hours

consumer.subscribe('tweets-lambda1')

In [35]:
for message in consumer:
    print(message.value.decode())

1102340628246077440;2019-09-27 08:27:33;;
1241258612;2019-09-27 08:27:07;London;
32183561;2019-09-27 08:27:07;Australia 🇦🇺;
2764956929;2019-09-27 08:27:04;South Africa;
166388185;2019-09-27 08:26:36;Australia;
4611042555;2019-09-27 08:26:34;;
56682701;2019-09-27 08:26:27;Naperville;
1251358218;2019-09-27 08:26:25;Nairobi,Kenya;
434807646;2019-09-27 08:26:20;Pakistan;
286281064;2019-09-27 08:26:09;Canada,;
6484062;2019-09-27 08:25:53;UC3M, Madrid;
4820804277;2019-09-27 08:25:43;;
2959060847;2019-09-27 08:25:39;Comunidad de Madrid, España;
833068479410417664;2019-09-27 08:25:38;Australia;
1111987907445882880;2019-09-27 08:24:57;Ile-de-France, France;
1496642287;2019-09-27 08:29:14;Earth;
215769483;2019-09-27 08:28:53;Nairobi,kenya.;
981090058831826945;2019-09-27 08:28:53;;
3869640735;2019-09-27 08:28:48;;
1057557859674800128;2019-09-27 08:28:32;Noida, India;
952128394900180992;2019-09-27 08:28:29;Developing Countries;
768435897071181824;2019-09-27 08:28:10;France;
800184230785363968;2019